Passive Learning with M=50

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

X_y_train = pd.read_csv('Bank_Train.csv',index_col=0)
X_y_test = pd.read_csv('Bank_Test.csv',index_col=0)
X_test = X_y_test.drop(columns=['y'])
y_test = X_y_test['y']
C_range = np.array([10**(-3),10**(-2),10**(-1),10**(0),10**(1),10**(2),10**(3),10**(4),10**(5),10**(6)])
Error_Test_df = pd.DataFrame(index=np.arange(1,51),columns=np.arange(10,910,10))

In [11]:
for mc in np.arange(1,51):    
    
    print(mc)     # Status Check
    
    X_y_train_updated = X_y_train.copy()

    X_y_train_build = pd.DataFrame()                     




    for i in np.arange(1,91):          

        if i != 90:
            
            X_y_train_to_add, X_y_train_updated = train_test_split(X_y_train_updated,train_size=10,shuffle=True,stratify=X_y_train_updated['y'])
            
            X_y_train_to_add.reset_index(drop=True,inplace=True)
            X_y_train_updated.reset_index(drop=True,inplace=True)

        else:
            X_y_train_to_add = X_y_train_updated



        X_y_train_build = pd.concat([X_y_train_build,X_y_train_to_add],axis=0)

        X_y_train_build.reset_index(drop=True,inplace=True)



        CV_errors_df = pd.DataFrame(index=np.arange(0,5),columns=C_range) 

        cv_ite = -1

        X_train_build = X_y_train_build.drop(columns=['y'])
        y_train_build = X_y_train_build['y']


        kf = StratifiedKFold(n_splits=5,shuffle=True)       


        for train,test in kf.split(X_train_build,y_train_build):

            cv_ite = cv_ite + 1

            X_train_cv, X_test_cv = X_train_build.iloc[train,:],X_train_build.iloc[test,:]
            y_train_cv, y_test_cv = y_train_build.iloc[train],y_train_build.iloc[test]


            for j in np.arange(0,C_range.size):

                svm_cv = LinearSVC(penalty='l1',dual=False,C=C_range[j]).fit(X_train_cv,y_train_cv)

                y_test_cv_predicted = svm_cv.predict(X_test_cv)

                mis_loc = 0

                for l in np.arange(0,X_test_cv.shape[0]):

                    if y_test_cv_predicted[l] != y_test_cv.iloc[l]:

                        mis_loc = mis_loc + 1

                err_loc = (mis_loc/X_test_cv.shape[0])*100

                CV_errors_df.iloc[cv_ite,j] = err_loc



        CV_mean_errors = CV_errors_df.mean(axis=0)

        error_vector = CV_mean_errors.to_numpy().flatten()

        indx_C_selected = np.argmin(error_vector)

        min_C = C_range[indx_C_selected]



        svm_f = LinearSVC(penalty='l1',dual=False,C=min_C).fit(X_train_build,y_train_build)

        y_test_predicted = svm_f.predict(X_test)

        mis = 0

        for k in np.arange(0,X_test.shape[0]):

            if y_test_predicted[k] != y_test.iloc[k]:

                mis = mis + 1

        err = (mis/X_test.shape[0])*100


        Error_Test_df.iloc[mc-1,i-1] = err





print(Error_Test_df)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
         10       20       30        40        50        60        70   \
1     9.5339  8.89831  9.95763   10.3814   5.08475   1.05932   1.05932   
2     2.9661   2.9661   2.9661    2.9661   2.75424   2.75424   2.75424   
3    26.6949  9.11017  2.75424   2.33051   1.69492   2.11864   2.33051   
4    4.02542  3.38983  2.75424   2.11864    2.9661   4.23729   3.60169   
5    12.9237  3.38983  3.38983   3.60169    2.9661   3.17797   2.11864   
6    10.5932  11.4407  4.23729   3.60169   1.90678   2.33051   2.11864   
7    27.7542   17.161  9.95763   8.68644   4.02542   1.90678   1.90678   
8    4.87288  4.87288  1.90678   1.90678   3.17797    2.9661    2.9661   
9    2.11864  2.54237  3.60169   1.05932   1.05932   2.54237   1.05932   
10   22.4576   2.9661   5.9322    2.9661   1.48305   1.90678   2.75424   
11   11.6525  17.3729  1.69492   1.27119   5.

In [12]:
Error_Test_df.to_csv('Passive_Learning_Errors.csv')